## Potential Pipeline Process

-User selects city
--test with TUcson and Ann Arbor, later add other cities 
-Use selection to ID building footprints and calculate available sq ft, add to geodataframe (gdf)
--total area can be added to City Specs if able to classify R/C

In [ ]:
# Imports
import numpy as np
from shapely.geometry import Polygon
import osmnx as ox
import geopandas as gpd

In [ ]:
# Section 1 - specify city and get building footprints and area

def fetch_buildings(city_name):
    tags = {"building": True}
    gdf = ox.features_from_place(city_name, tags=tags)

    # Keep only polygons
    gdf = gdf[gdf.geometry.type.isin(["Polygon", "MultiPolygon"])]

    # Reproject to metric (for area calculations)
    gdf = gdf.to_crs(epsg=3857)
    
    # Add area in m²
    gdf["area_m2"] = gdf.geometry.area

    # Convert back to WGS84 for mapping
    gdf = gdf.to_crs(epsg=4326)

    return gdf

In [ ]:
# Section 2 - what is this for???

def polygon_orientation(polygon: Polygon):
    if polygon.is_empty:
        return np.nan
    
    # Minimum rotated rectangle
    mrr = polygon.minimum_rotated_rectangle
    coords = list(mrr.exterior.coords)

    # Longest edge
    max_len = 0
    best_angle = None
    
    for i in range(len(coords)-1):
        (x1, y1), (x2, y2) = coords[i], coords[i+1]
        dx, dy = x2 - x1, y2 - y1
        edge_len = np.hypot(dx, dy)
        
        if edge_len > max_len:
            max_len = edge_len
            best_angle = np.degrees(np.arctan2(dy, dx))
    
    # Normalize
    if best_angle < 0:
        best_angle += 360

    return best_angle

In [ ]:
# Section 3 - get roof tilt 
def estimate_tilt(gdf):
    # Height available?
    if "height" in gdf.columns:
        heights = pd.to_numeric(gdf["height"], errors="coerce")
    elif "building:levels" in gdf.columns:
        heights = pd.to_numeric(gdf["building:levels"], errors="coerce") * 3.2
    else:
        heights = 5  # fallback
    
    # Building footprint width (effective roof span)
    gdf["width_m"] = gdf.geometry.apply(lambda x: x.minimum_rotated_rectangle.length / 2)

    # Tilt = arctan(height / width)
    gdf["tilt_deg"] = np.degrees(np.arctan(heights / gdf["width_m"]))

    # Clamp tilt for flat roofs
    gdf.loc[gdf["tilt_deg"] < 3, "tilt_deg"] = 3
    
    return gdf

In [ ]:
# Section 4 - roof shade (nice to have, not need to have)

import rasterio
from rasterio.mask import mask

def shade_from_geotiff(raster, polygon):
    # Mask satellite raster to building footprint
    out_img, out_transform = mask(raster, [polygon], crop=True)
    arr = out_img[0]

    # Normalize
    arr = (arr - arr.min()) / (arr.max() - arr.min() + 1e-6)

    # Shade = lower brightness
    shade_score = 1 - arr.mean()  # 0 = bright, 1 = dark/shaded
    
    return shade_score

def add_shade_scores(gdf, raster):
    gdf["shade"] = gdf.geometry.apply(lambda poly: shade_from_geotiff(raster, poly))
    return gdf

In [ ]:
# Section 5 - compute city irradiance from azimuth values
# only works with our pre-defined tables in Supabase, can't do for other cities

def compute_city_irradiance(df):
    daylight_hours = (df["sunset"] - df["sunrise"]).mean()
    azimuth_range = (df["azimuth_sunset"] - df["azimuth_sunrise"]).mean()

    # Normalize both to 0–1
    daylight_score = daylight_hours / 15  
    azimuth_score = azimuth_range / 180

    return 0.7*daylight_score + 0.3*azimuth_score

In [ ]:
# Section 6

def orientation_match(roof_angle, sun_angle=180):
    diff = abs(roof_angle - sun_angle)
    if diff > 180:
        diff = 360 - diff
    return 1 - diff / 180

In [ ]:
# Section 7 - Compute 'solar score' ranking 

def compute_solar_score(gdf, irradiance_factor):
    orientation_score = gdf["orientation_deg"].apply(lambda x: orientation_match(x))
    tilt_score = 1 - abs(gdf["tilt_deg"] - 25) / 45  # optimal 20–30°
    shade_score = 1 - gdf["shade"]  # invert shade

    gdf["solar_score"] = (
        0.35 * orientation_score +
        0.25 * tilt_score +
        0.25 * shade_score +
        0.15 * irradiance_factor
    )

    # Clamp
    gdf["solar_score"] = gdf["solar_score"].clip(0, 1)

    return gdf

In [ ]:
# Section 8 - putting all the data together

def build_solar_model(city_name, demand_df, raster):
    buildings = fetch_buildings(city_name)
    buildings = add_orientation(buildings)
    buildings = estimate_tilt(buildings)
    buildings = add_shade_scores(buildings, raster)

    irradiance_factor = compute_city_irradiance(demand_df)

    buildings = compute_solar_score(buildings, irradiance_factor)

    return buildings

In [ ]:
# Run model
best = buildings[buildings["solar_score"] > 0.8]

# Export data to Supabase for easy recall
json_data = buildings.to_json()

supabase.table("building_suitability").insert({"city": city_name, "geojson": json_data}).execute()